In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from aicsimageio import AICSImage
import time

### package for 3d visualization
#from itkwidgets import view                              
#from aicssegmentation.core.visual import seg_fluo_side_by_side,  single_fluorescent_view, segmentation_quick_view
#import skimage

### local new python segmentation functions
import os
import sys
pythonPackagePath = os.path.abspath('../src/')
sys.path.append(pythonPackagePath)
from peak_local_max_3d import peak_local_max_3d 
from gaussian_fitting import fit_multiple_gaussians
from extract_data import extract_data_from_filename
from gaussian_visualization import visualize_3D_gaussians
from gaussian_fitting import check_fitting_error
import matplotlib.pyplot as plt
from skimage import io
import tifffile
import zarr

In [2]:
z2 = zarr.open('/Users/apple/Desktop/Akamatsu_Lab/Data/zarr_data/Channel_3/transpose-148f28e615c88260294c4c8208b84ca3', mode='r')

save_dir = '/Users/apple/Desktop/Akamatsu_Lab/Data/Channel_3_pkl_zarr'

In [3]:
z2.info

Type,zarr.core.Array
Data type,uint16
Shape,"(130, 1, 116, 2052, 340)"
Chunk shape,"(1, 1, 116, 2052, 340)"
Order,C
Read-only,True
Compressor,"Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0)"
Store type,zarr.storage.DirectoryStore
No. bytes,21042028800 (19.6G)
No. bytes stored,4452019926 (4.1G)
Storage ratio,4.7


In [4]:
z2.shape

(130, 1, 116, 2052, 340)

In [5]:
frames = z2.shape[0]
print(f'the number of frames are {frames}')

the number of frames are 130


In [6]:
import time

# Load the CZI file
#img = AICSImage(input_filename_czi)

#get the number of frames for our original data for automated analysis for all frames 
frames = z2.shape[0]
start_frame = 0
end_frame = frames

start_time = time.time()

for frame in range(start_frame,end_frame):
    
    print('frame number is', frame)
    
    single_frame_input = z2[frame]
    single_frame_input = single_frame_input[0,:,:,:]
    print(single_frame_input.shape)
    single_frame_input = np.transpose(single_frame_input,axes =(0,2,1))
    print(single_frame_input.shape)

    #define threshold: a value(intensity) for the pixel below which all values would be considered noise and dropped 
    #define min_distance: min_distance/2 is the radius within which we will keep the peak with max value/intensity or 
    #if two peaks have the same value they will be kept 
    
    maximas = peak_local_max_3d(single_frame_input,min_distance=10,threshold=180)
    print('local_maximas detected are', maximas.shape[0])


    #give the expected std dev/radius of our particles for x,y,z 
    sigmaExpected_x__pixels = 2
    sigmaExpected_y__pixels = 2
    sigmaExpected_z__pixels = 4

    sigmas_guesses = []
    for i in range(len(maximas)):
        sigmas_guesses.append([sigmaExpected_z__pixels,sigmaExpected_x__pixels,sigmaExpected_y__pixels])
        
    #last parameter in the fit_multiple_gaussians is similar to min_distance above, we should give half of the 
    #value here of min_distance   
    gaussians, gaussians_popt = fit_multiple_gaussians(single_frame_input,maximas,sigmas_guesses,5)
        
    accumulator = []
    for gaussian in gaussians:

        if(gaussian!=-1):
            amplitude = gaussian[0]

            #print(gaussian)
            mu_x     = int(gaussian[1][1]) ##this is going to be mu_z, previous code [1][0]
            mu_y     = int(gaussian[1][2]) ##need to finalise what this is (x or y) [1][1]
            mu_z     = int(gaussian[1][0]) ##need to finalise what this is (x or y) [1][2]
            ##sigmas will also change due to the above 
            sigma_x  = int(gaussian[2][1]) 
            sigma_y  = int(gaussian[2][2])
            sigma_z  = int(gaussian[2][0])
            accumulator.append(np.array([amplitude,mu_x,mu_y,mu_z,sigma_x,sigma_y,sigma_z]))
            
    accumulator = np.array(accumulator)
    print(accumulator.shape)
    df = pd.DataFrame()
    df['amplitude'] = accumulator[:,0]
    df['mu_x'] = accumulator[:,1]
    df['mu_y'] = accumulator[:,2]
    df['mu_z'] = accumulator[:,3]
    df['sigma_x'] = accumulator[:,4]
    df['sigma_y'] = accumulator[:,5]
    df['sigma_z'] = accumulator[:,6]
    df.head()
    
    error_list, index_list = check_fitting_error(single_frame_input,maximas,gaussians,sigmas_guesses)
    
    # Construct the filename
    '''
    filename_csv = f'df_c2_t{frame}.csv'
    file_path_csv = os.path.join(csv_save_dir, filename_csv)
    df.to_csv(file_path_csv)
    '''

    # Construct the filename based on the loop index (time_frame)
    filename_pkl = f'df_c3_t{frame}.pkl'

    # Construct the full file path by joining the directory and filename
    file_path = os.path.join(save_dir, filename_pkl)

    # Save the DataFrame to a pickle file with the specified path
    df.to_pickle(file_path)
end_time = time.time()

print('time taken (seconds)', end_time - start_time)

frame number is 0
(116, 2052, 340)
(116, 340, 2052)
local_maximas detected are 6200
10%(620 of 6200)
20%(1240 of 6200)
30%(1860 of 6200)
40%(2480 of 6200)
50%(3100 of 6200)
60%(3720 of 6200)
70%(4340 of 6200)
80%(4960 of 6200)
90%(5580 of 6200)
100%(6200 of 6200)
(6200, 7)
the number of times the gaussian fitting worked was6200and the number of times the gaussian did not fit was 0
frame number is 1
(116, 2052, 340)
(116, 340, 2052)
local_maximas detected are 6067
10%(607 of 6067)
20%(1214 of 6067)
30%(1821 of 6067)
40%(2427 of 6067)
50%(3034 of 6067)
60%(3641 of 6067)
70%(4247 of 6067)
80%(4854 of 6067)
90%(5461 of 6067)
100%(6067 of 6067)
(6067, 7)
the number of times the gaussian fitting worked was6067and the number of times the gaussian did not fit was 0
frame number is 2
(116, 2052, 340)
(116, 340, 2052)
local_maximas detected are 6157
10%(616 of 6157)
20%(1232 of 6157)
30%(1848 of 6157)
40%(2463 of 6157)
50%(3079 of 6157)
60%(3695 of 6157)
70%(4310 of 6157)
80%(4926 of 6157)
90%(5